In [0]:
%run

"../includes/configuration"

In [0]:
race_results_df = spark.read.parquet(f"{presentation_folder_path}/race_results/")

race_results_df.show()

+---------+--------------------+-------------------+----------------+--------------------+-------------+------------------+-----------+----+-----------+-----------+------+--------+--------------------+
|race_year|           race_name|          race_date|circuit_location|         driver_name|driver_number|driver_nationality|       team|grid|fastest_lap|  race_time|points|position|        created_date|
+---------+--------------------+-------------------+----------------+--------------------+-------------+------------------+-----------+----+-----------+-----------+------+--------+--------------------+
|     2009|Australian Grand ...|2009-03-29 06:00:00|       Melbourne|      Lewis Hamilton|           44|           British|    McLaren|  18|         39|         \N|   0.0|    NULL|2023-10-06 16:53:...|
|     2009|Australian Grand ...|2009-03-29 06:00:00|       Melbourne|   Heikki Kovalainen|         NULL|           Finnish|    McLaren|  12|       NULL|         \N|   0.0|    NULL|2023-10-06 1

In [0]:
from pyspark.sql.functions import count, sum, when, col

driver_standings_df = race_results_df \
.groupBy("race_year", "driver_name", "driver_nationality", "team") \
.agg(sum("points").alias("total_points"),
     count(when(col("position") == 1, True)).alias("wins"))

In [0]:
display(driver_standings_df.where("race_year = 2020"))

race_year,driver_name,driver_nationality,team,total_points,wins
2020,Lance Stroll,Canadian,Racing Point,75.0,0
2020,Kevin Magnussen,Danish,Haas F1 Team,1.0,0
2020,Antonio Giovinazzi,Italian,Alfa Romeo,4.0,0
2020,Carlos Sainz,Spanish,McLaren,105.0,0
2020,Nicholas Latifi,Canadian,Williams,0.0,0
2020,Lewis Hamilton,British,Mercedes,347.0,11
2020,Pierre Gasly,French,AlphaTauri,75.0,1
2020,Jack Aitken,British,Williams,0.0,0
2020,Romain Grosjean,French,Haas F1 Team,2.0,0
2020,Daniil Kvyat,Russian,AlphaTauri,32.0,0


In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import rank, desc

In [0]:
driver_rank_spec = Window.partitionBy("race_year").orderBy(desc("total_points"), desc("wins"))

final_df = driver_standings_df.withColumn("rank", rank().over(driver_rank_spec))

In [0]:
display(final_df.where("race_year = 2020"))

race_year,driver_name,driver_nationality,team,total_points,wins,rank
2020,Lewis Hamilton,British,Mercedes,347.0,11,1
2020,Valtteri Bottas,Finnish,Mercedes,223.0,2,2
2020,Max Verstappen,Dutch,Red Bull,214.0,2,3
2020,Sergio Pérez,Mexican,Racing Point,125.0,1,4
2020,Daniel Ricciardo,Australian,Renault,119.0,0,5
2020,Carlos Sainz,Spanish,McLaren,105.0,0,6
2020,Alexander Albon,Thai,Red Bull,105.0,0,6
2020,Charles Leclerc,Monegasque,Ferrari,98.0,0,8
2020,Lando Norris,British,McLaren,97.0,0,9
2020,Pierre Gasly,French,AlphaTauri,75.0,1,10


In [0]:
# final_df.write.parquet(f"{presentation_folder_path}/driver_standings", mode="overwrite")

final_df.write.format("parquet").mode("overwrite").saveAsTable("f1_presentation.driver_standings")

In [0]:
%sql
SELECT *
FROM f1_presentation.driver_standings

race_year,driver_name,driver_nationality,team,total_points,wins,rank
1950,Nino Farina,Italian,Alfa Romeo,30.0,3,1
1950,Luigi Fagioli,Italian,Alfa Romeo,28.0,0,2
1950,Juan Fangio,Argentine,Alfa Romeo,27.0,3,3
1950,Louis Rosier,French,Talbot-Lago,13.0,0,4
1950,Alberto Ascari,Italian,Ferrari,11.0,0,5
1950,Johnnie Parsons,American,Kurtis Kraft,9.0,1,6
1950,Bill Holland,American,Deidt,6.0,0,7
1950,Prince Bira,Thai,Maserati,5.0,0,8
1950,Peter Whitehead,British,Ferrari,4.0,0,9
1950,Louis Chiron,Monegasque,Maserati,4.0,0,9
